# Project Fungarium-Terrarium Technical Document
# Interphases with sensors with calibration and data acquisition

## Introduction

The have decided to use the ESP32-Camera Module as the microcontroller. <br>

<img src='https://cdn.shopify.com/s/files/1/1509/1638/products/ESP32-Cam_Modul_ESP32_WifiBluetooth_Modul_inklusive_Kamera_1er_without_logo_700x.jpg?v=1591264489' alt='ESP32-Cam' width='300' />

Advantages:
* Cheaper than Raspberry Pi
* Low Energy
* Easy management

Disadvantages: 
* Less pins
* Lower image quality

### A bit into the ESP-32-Cam

Here are the functional units. 

<img src='https://www.mischianti.org/wp-content/uploads/2020/06/Function-Block-diagram-Espressif-esp32-Wi-Fi-Bluetooth-Microcontroller.jpg' alt='func_units_esp32' width='500'/>

* Processors:
    * CPU: Xtensa dual-core 32-bit LX6 microprocessor, operating at 240 MHz and performing at up to 600 DMIPS
    * Ultra low power (ULP) co-processor
    * Memory: 520 KiB SRAM, external 4M PSRAM
* Wireless connectivity:
    * Wi-Fi: 802.11 b/g/n
    * Bluetooth: v4.2 BR/EDR and BLE (shares the radio with Wi-Fi)
    * Support OV2640 and OV7670 cameras, built-in flash
    * Support TF card
* Security:
    * IEEE 802.11 standard security features all supported, including WFA, WPA/WPA2 and WAPI
    * Secure boot
    * Flash encryption
    * 1024-bit OTP, up to 768-bit for customers
    * Cryptographic hardware acceleration: AES, SHA-2, RSA, elliptic curve cryptography (ECC), random number generator (RNG)
* Power management:
    * Internal low-dropout regulator
    * Individual power domain for RTC
    * 5μA deep sleep current
    * Wake up from GPIO interrupt, timer, ADC measurements, capacitive touch sensor interrupt



### Pinout distribution

<img src='https://www.mischianti.org/wp-content/uploads/2020/09/ESP32-CAM-pinout-mischianti.jpg' alt='pinout_esp32' width='700' />

Now we can see which are out options for the sensors and camera. 


### Powering the ESP32

This device doesn’t have an installed USB to TTL converter, so you need an  FTDI programmer. As we look to have a source and backup power. We would like to have this configuration. <br>

<img src='https://www.mischianti.org/wp-content/uploads/2021/03/esp32-cam-upload-sketch-and-normal-connection-schema-5v-external-power.jpg' alt='power_dist_diag' width='450'/>

If the power isn’t sufficient, you receive a ``“Brownout detector was triggered“``

|Operation mode|	Power|
|--------------|:--------:|
|Stand by	|  80mHa|
|In streaming |	100~160mAh|
|In streaming with flash |	270mAh|

___ Where can we find the FTDIs and the external energy modules? ___


### Configuring the IDE

Using the Arduino IDE <br>
Given its versatility and its vast array of pre-coded snippets with sensors in the market. We will use the arduino IDE as the main interphase for the ESP32.<br>
The link is [here](https://www.arduino.cc/en/software) <br>
<img src='https://www.arduino.cc/wiki/370832ed4114dd35d498f2f449b4781e/arduino.svg' alt='icon_arduinoIDE' width='300'/>

First, you must add esp32 URL descriptor to your IDE
https://dl.espressif.com/dl/package_esp32_index.json
Go to File –> Preferences and add the URL on “Additional Boards Manager URLs.

<img src='https://www.mischianti.org/wp-content/uploads/2020/05/Arduino-IDE-esp32-additional-board-manager.jpg' atl='screenshot1' width='500'/>

find and upload the board

<img src='https://www.mischianti.org/wp-content/uploads/2020/09/ArduinoIDE-select-ESP32-CAM-AI-Thinker.jpg' alt='screenshot2' width='500'/>









### libraries for main_ code

The file we are refering to can be found here (make hyperlink later (main_.h)).

If you buy generic ESP32-CAM on the Chinese market or from some supplier, you will probably have to change the configuration in this way.

First, you must uncomment the correct clone model as in the upper instructions:

`` #Select camera model``<br>
``#define CAMERA_MODEL_WROVER_KIT``<br>
``#define CAMERA_MODEL_ESP_EYE``<br>
``#define CAMERA_MODEL_M5STACK_PSRAM``<br>
``#define CAMERA_MODEL_M5STACK_WIDE``<br>
``#define CAMERA_MODEL_AI_THINKER ``<br>

After loading, you may get an error in the serial monitor with this message, __which means that the power supply you use offers enough power.__

you can try to connect to an external power supply or add this line to the setup with the correct include

`` #include "soc/soc.h"``<br>
`` #include "soc/rtc_cntl_reg.h" `` <br>

The main configuration of the board is made in the ``setup()`` section:<br>

`` setup() {``<br>
    ``...``<br>
    ``WRITE_PERI_REG(RTC_CNTL_BROWN_OUT_REG, 0); //disable brownout detector`` <br>





### setup definitions

We will take sometime here to explain the setup if the microcontroller and check how to use and define each pinout. <br>

```{
void setup() {
  WRITE_PERI_REG(RTC_CNTL_BROWN_OUT_REG, 0); //disable brownout detector
 
  Serial.begin(115200);
  Serial.setDebugOutput(true);
  Serial.println();
 
  camera_config_t config;
  config.ledc_channel = LEDC_CHANNEL_0;
  config.ledc_timer = LEDC_TIMER_0;
  config.pin_d0 = Y2_GPIO_NUM;
  config.pin_d1 = Y3_GPIO_NUM;
  config.pin_d2 = Y4_GPIO_NUM;
  config.pin_d3 = Y5_GPIO_NUM;
  config.pin_d4 = Y6_GPIO_NUM;
  config.pin_d5 = Y7_GPIO_NUM;
  config.pin_d6 = Y8_GPIO_NUM;
  config.pin_d7 = Y9_GPIO_NUM;
  config.pin_xclk = XCLK_GPIO_NUM;
  config.pin_pclk = PCLK_GPIO_NUM;
  config.pin_vsync = VSYNC_GPIO_NUM;
  config.pin_href = HREF_GPIO_NUM;
  config.pin_sscb_sda = SIOD_GPIO_NUM;
  config.pin_sscb_scl = SIOC_GPIO_NUM;
  config.pin_pwdn = PWDN_GPIO_NUM;
  config.pin_reset = RESET_GPIO_NUM;
  config.xclk_freq_hz = 20000000;
  config.pixel_format = PIXFORMAT_JPEG;
  //init with high specs to pre-allocate larger buffers
  if(psramFound()){
    config.frame_size = FRAMESIZE_UXGA;
    config.jpeg_quality = 10;
    config.fb_count = 2;
  } else {
    config.frame_size = FRAMESIZE_SVGA;
    config.jpeg_quality = 12;
    config.fb_count = 1;
  }
}
``` 

this can be directly related to 

<img src='https://www.mischianti.org/wp-content/uploads/2020/09/ESP32-CAM-pinout-mischianti.jpg' alt='pinout_esp32' width='500' />




## Looking at the sensors and their basic code

We now see a description of the different sensors.<br>

### Sensirion SCD30 CO2_Humidity_Temperature

<img src='https://sensirion.com/images/scale/600x375/com-sensirion-master/portfolio/series/image/7b0bf344-1997-4444-8217-f23a1bf10868.png' alt ='scd_30' width='300'/>

The main documentation for the sensor can be found [here](https://sensirion.com/de/produkte/katalog/SCD30/)

The simplest script, with a general overview goes as:

```
#include "scd30.h"  \\ necesary library

#include <stdio.h> /* printf */

/* TO USE CONSOLE OUTPUT (printf) YOU MAY NEED TO ADAPT THE
 * INCLUDE ABOVE OR DEFINE IT ACCORDING TO YOUR PLATFORM.
 * #define printf(...)
 */

int main(void) {
    float co2_ppm, temperature, relative_humidity;
    int16_t err;
    uint16_t interval_in_seconds = 2;

    /* Initialize I2C */
    sensirion_i2c_init();

    /* Busy loop for initialization, because the main loop does not work without
     * a sensor.
     */
    while (scd30_probe() != NO_ERROR) {
        printf("SCD30 sensor probing failed\n");
        sensirion_sleep_usec(1000000u);
    }
    printf("SCD30 sensor probing successful\n");

    scd30_set_measurement_interval(interval_in_seconds);
    sensirion_sleep_usec(20000u);
    scd30_start_periodic_measurement(0);

    while (1) {
        uint16_t data_ready = 0;
        uint16_t timeout = 0;

        /* Poll data_ready flag until data is available. Allow 20% more than
         * the measurement interval to account for clock imprecision of the
         * sensor.
         */
        for (timeout = 0; (100000 * timeout) < (interval_in_seconds * 1200000);
             ++timeout) {
            err = scd30_get_data_ready(&data_ready);
            if (err != NO_ERROR) {
                printf("Error reading data_ready flag: %i\n", err);
            }
            if (data_ready) {
                break;
            }
            sensirion_sleep_usec(100000);
        }
        if (!data_ready) {
            printf("Timeout waiting for data_ready flag\n");
            continue;
        }

        /* Measure co2, temperature and relative humidity and store into
         * variables.
         */
        err =
            scd30_read_measurement(&co2_ppm, &temperature, &relative_humidity);
        if (err != NO_ERROR) {
            printf("error reading measurement\n");

        } else {
            printf("measured co2 concentration: %0.2f ppm, "
                   "measured temperature: %0.2f degreeCelsius, "
                   "measured humidity: %0.2f %%RH\n",
                   co2_ppm, temperature, relative_humidity);
        }
    }

    scd30_stop_periodic_measurement();
    return 0;
}
```


### Seeed Grove SCD30 CO2_Humidity_Temperature 

This is the same sensor as the later one, exactly the same.<br>
In this case we are looking for the change the casing does to the measurement. <br>

<img src='https://files.seeedstudio.com/wiki/Grove-CO2-Temperature-Humidity-Sensor-SCD30/img/main.jpg' atl='grove_scd30' width='400'/>

The calibration procedure can be done the same way as well as data aquisition. <br>

The information about the sensor can be found [here](https://wiki.seeedstudio.com/Grove-CO2_Temperature_Humidity_Sensor-SCD30/) <br>

its sample script goes as follows:

```
#include "SCD30.h"
 
#if defined(ARDUINO_ARCH_AVR)
    #pragma message("Defined architecture for ARDUINO_ARCH_AVR.")
    #define SERIAL Serial
#elif defined(ARDUINO_ARCH_SAM)
    #pragma message("Defined architecture for ARDUINO_ARCH_SAM.")
    #define SERIAL SerialUSB
#elif defined(ARDUINO_ARCH_SAMD)
    #pragma message("Defined architecture for ARDUINO_ARCH_SAMD.")  
    #define SERIAL SerialUSB
#elif defined(ARDUINO_ARCH_STM32F4)
    #pragma message("Defined architecture for ARDUINO_ARCH_STM32F4.")
    #define SERIAL SerialUSB
#else
    #pragma message("Not found any architecture.")
    #define SERIAL Serial
#endif
 
 
 
void setup()
{
    Wire.begin();
    SERIAL.begin(115200);
    SERIAL.println("SCD30 Raw Data");
    scd30.initialize();
}
 
void loop()
{
    float result[3] = {0};
 
    if(scd30.isAvailable())
    {
        scd30.getCarbonDioxideConcentration(result);
        SERIAL.print("Carbon Dioxide Concentration is: ");
        SERIAL.print(result[0]);
        SERIAL.println(" ppm");
        SERIAL.println(" ");
        SERIAL.print("Temperature = ");
        SERIAL.print(result[1]);
        SERIAL.println(" ℃");
        SERIAL.println(" ");
        SERIAL.print("Humidity = ");
        SERIAL.print(result[2]);
        SERIAL.println(" %");
        SERIAL.println(" ");
        SERIAL.println(" ");
        SERIAL.println(" ");
    }
 
    delay(2000);
}
```


### Multichannel Gas Sensor

This is a semi-reliable sensor for gases such as:
* Carbon Monoxide ($CO$)
* Nitrogen Dioxide ($NO_2$)
* Ethyl Alcohol ($C_2H_5CH$)
* and other organic volatile compounds (VOCs)

<img src='https://files.seeedstudio.com/wiki/Grove-Multichannel_Gas_Sensor/img/Grove-Multichannel_Gas_Sensor_V2_101020820/IMG/04.png' alt='VOCs_sensor' width='400'/>

The documentation for the sensor is [here](https://wiki.seeedstudio.com/Grove-Multichannel-Gas-Sensor-V2/)<br>

The main code is here:

```
#include <TFT_eSPI.h>
#include <Multichannel_Gas_GMXXX.h>
#include <Wire.h>
GAS_GMXXX<TwoWire> gas;
 
TFT_eSPI tft; 
// Stock font and GFXFF reference handle
TFT_eSprite spr = TFT_eSprite(&tft);  // Sprite 
 
void setup() {
  // put your setup code here, to run once:
  tft.begin();
  tft.setRotation(3);
  spr.createSprite(tft.width(),tft.height()); 
  gas.begin(Wire, 0x08); // use the hardware I2C
}
 
void loop() {
  // put your main code here, to run repeatedly:
  int val;
  spr.fillSprite(TFT_BLACK);
  spr.setFreeFont(&FreeSansBoldOblique18pt7b); 
  spr.setTextColor(TFT_BLUE);
  spr.drawString("Gas Terminal", 60 - 15, 10 , 1);// Print the test text in the custom font
  for(int8_t line_index = 0;line_index < 5 ; line_index++)
  {
    spr.drawLine(0, 50 + line_index, tft.width(), 50 + line_index, TFT_GREEN);
  }
 
  spr.setFreeFont(&FreeSansBoldOblique9pt7b);                 // Select the font
  // GM102B NO2 sensor
  val = gas.getGM102B();
  if (val > 999) val = 999;
  spr.setTextColor(TFT_WHITE);
  spr.drawString("NO2:", 60 - 24, 100 -24 , 1);// Print the test text in the custom font
  spr.drawRoundRect(60 - 24,100,80,40,5,TFT_WHITE); 
  spr.setTextColor(TFT_WHITE);
  spr.drawNumber(val,60 - 20,100+10,1);
  spr.setTextColor(TFT_GREEN);
  spr.drawString("ppm", 60 + 12, 100+8, 1);
  // GM302B C2H5CH sensor
  val = gas.getGM302B();
  if (val > 999) val = 999;
  spr.setTextColor(TFT_WHITE);
  spr.drawString("C2H5CH:", 230 -24 , 100 - 24 , 1);// Print the test text in the custom font
  spr.drawRoundRect(230 - 24,100,80,40,5,TFT_WHITE);
  spr.setTextColor(TFT_WHITE);
  spr.drawNumber(val,230 - 20,100+10,1);
  spr.setTextColor(TFT_GREEN);
  spr.drawString("ppm", 230 + 12, 100+8, 1);
  // GM502B VOC sensor
  val = gas.getGM502B();
  if (val > 999) val = 999;
  spr.setTextColor(TFT_WHITE);
  spr.drawString("VOC:", 60 - 24, 180 -24 , 1);// Print the test text in the custom font
  spr.drawRoundRect(60 - 24,180,80,40,5,TFT_WHITE);
  spr.setTextColor(TFT_WHITE);
  spr.drawNumber(val,60 - 20,180+10,1);
  spr.setTextColor(TFT_GREEN);
  spr.drawString("ppm", 60 + 12, 180+8, 1);
  // GM702B CO sensor
  val = gas.getGM702B();
  if (val > 999) val = 999;
  spr.setTextColor(TFT_WHITE);
  spr.drawString("CO:", 230 -24 , 180 - 24, 1);// Print the test text in the custom font
  spr.drawRoundRect(230 - 24 ,180,80,40,5,TFT_WHITE);
  spr.setTextColor(TFT_WHITE);
  spr.drawNumber(val ,230 - 20 ,180+10,1);
  spr.setTextColor(TFT_GREEN);
  spr.drawString("ppm", 230 + 12, 180+8, 1);
 
  spr.pushSprite(0, 0);
  delay(100);
 
}
```




### Capacitive Soil Moisture Sensor

It is a very popular sensor which is WAY more efficient that its resistive counterpart.<br>
Its is an analog sensor, with plenty of posibilities. <br>
Requiring a 5V each and we plan for 3 of them. <br>

<img src='https://files.seeedstudio.com/wiki/Grove-Capacitive_Moisture_Sensor_Corrosion_Resistant/img/line.jpg' alt='sensor_levels' width='400' />

Its is an analog sensor, with plenty of posibilities. <br>

<img src='https://esp32io.com/images/tutorial/esp32-soil-moisture-sensor-wiring-diagram.jpg' atl='Soil_Moisture_Sensor' width='400'/>

```
https://arduino.cc/en/Tutorial/AnalogReadSerial
*/


 
// the setup routine runs once when you press reset:
void setup() {
  // initialize serial communication at 9600 bits per second:
  Serial.begin(9600);
}
 
// the loop routine runs over and over again forever:
void loop() {
  // read the input on analog pin 0:
  int sensorValue = analogRead(A0);
  // print out the value you read:
  Serial.println(sensorValue);
  delay(100);        // delay in between reads for stability
}
```




### Relay Modules

The relay module (modules) will be used to control the pumps, LEDs and ventilarors. <br>

<img src='https://i0.wp.com/randomnerdtutorials.com/wp-content/uploads/2019/12/Relay-1-2-4-8-ch-channel-modules.jpg?w=750&quality=100&strip=all&ssl=1' atl='relay_module' width='400'/>

This is an analog on/off controler. <br>

Its connection goes as follows:<br>

<img src='https://i0.wp.com/randomnerdtutorials.com/wp-content/uploads/2019/12/relay-esp8266-wiring.png?w=1038&quality=100&strip=all&ssl=1' alt='Relay_connections' width='400'/>

<img src='https://i0.wp.com/randomnerdtutorials.com/wp-content/uploads/2019/12/ESP32-Relay-Module-Tutorial-project-example-off.jpg?w=750&quality=100&strip=all&ssl=1' alt='Relay_connections' width='400'/>

```

#define RELAY_PIN 16 // ESP32 pin GIOP16 connected to the IN pin of relay

pinMode(16, OUTPUT);
digitalWrite(16, LOW); // GIOP16
digitalWrite(16, HIGH); // GIOP16


// the code in setup function runs only one time when ESP32 starts
void setup() {
  // initialize digital pin as an output.
  pinMode(RELAY_PIN, OUTPUT);
}

// the code in loop function is executed repeatedly infinitely
void loop() {
  digitalWrite(RELAY_PIN, HIGH);
  delay(1000);
  digitalWrite(RELAY_PIN, LOW);
  delay(1000);
}
```


